This notebook provides examples to go along with the [textbook](http://manipulation.csail.mit.edu/robot.html).  I recommend having both windows open, side-by-side!

In [1]:
from pydrake.all import FindResourceOrThrow, ModelVisualizer, StartMeshcat
import numpy as np

In [2]:
# Start the visualizer.
meshcat = StartMeshcat()

INFO:drake:Meshcat listening for connections at http://localhost:7001


# Robot arms

The next two cells will give you a simple interface to move the joints around on a variety of robot arm models.

Have a favorite robot that I haven't included here?  If you send me a link to a properly licensed URDF or SDF description of the robot and it's mesh files, I'm happy to add it!  It's worth taking a look at the files quickly, to get a sense for how they work: [SDF](https://github.com/RobotLocomotion/drake/blob/master/manipulation/models/iiwa_description/sdf/iiwa14_no_collision.sdf), [URDF](https://github.com/RobotLocomotion/drake/blob/master/manipulation/models/iiwa_description/urdf/iiwa14_no_collision.urdf).

Note: The Jaco doesn't visualize properly in this renderer yet.  See drake issue [#13846](https://github.com/RobotLocomotion/drake/issues/13846).

In [7]:
# First pick your robot by un-commenting one of these:
robot = "Kuka LBR iiwa 7"
# robot = "Kuka LBR iiwa 14"
# robot = "Kinova Jaco Gen2 (7 DoF)"
# robot = "Franka Emika Panda"
# robot = "UR3e"


def get_model_file(description):
    # Note: I could download remote model resources here if necessary.
    # TODO(russt): Update this to AddModelsfromUrl pending resolution of
    # https://github.com/RobotLocomotion/drake/issues/18977
    if description == "Kuka LBR iiwa 7":
        return FindResourceOrThrow(
            "drake/manipulation/models/iiwa_description/iiwa7/iiwa7_no_collision.sdf"
        )
    elif description == "Kuka LBR iiwa 14":
        return FindResourceOrThrow(
            "drake/manipulation/models/iiwa_description/sdf/iiwa14_no_collision.sdf"
        )
    elif description == "Kinova Jaco Gen2 (7 DoF)":
        return FindResourceOrThrow(
            "drake/manipulation/models/jaco_description/urdf/j2s7s300.urdf"
        )
    elif description == "Franka Emika Panda":
        return FindResourceOrThrow(
            "drake/manipulation/models/franka_description/urdf/panda_arm_hand.urdf"
        )
    elif description == "UR3e":
        return FindResourceOrThrow(
            "drake/manipulation/models/ur3e/ur3e_cylinders_collision.urdf"
        )
    raise Exception("Unknown model")


visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.AddModels(get_model_file(robot))

visualizer.Run(loop_once=False)
meshcat.DeleteAddedControls()

Click 'Stop Running' or press Esc to quit


# Robot hands

I don't have the same smörgåsbord of robot models to offer for robot hands (yet).  We do have the allegro hand model available, and I'm happy to add more here (though many will require some care to handle their kinematic constraints).  For now, you can at least try out the [Allegro Hand](http://www.wonikrobotics.com/Allegro-Hand.htm).

TODO(russt): Consider porting Robotiq, Sandia, or IHY-hand from our [openhumanoids](https://github.com/openhumanoids/oh-distro/tree/master/software/models/common_components) project.

In [10]:
meshcat.Delete()

visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.AddModels(
    FindResourceOrThrow(
        "drake/manipulation/models/allegro_hand_description/sdf/allegro_hand_description_right.sdf"
    )
)

visualizer.Run(loop_once=False)

meshcat.DeleteAddedControls()

Click 'Stop Running' or press Esc to quit


# Cat URDF

In [3]:
def calculate_MOI_tensor(m, r, h, rotation_axis="x"):
    """
    https://en.wikipedia.org/wiki/Moment_of_inertia#Inertia_tensor
    Mass in kg
    Radius in m
    Height in m
    """
    axis_to_idx = {
        "x": 0,
        "y": 1,
        "z": 2,
    }

    MOI_off_axis = (1/12) * m * (3 * r**2 + h**2)
    MOI_on_axis = (1/2) * m * r**2

    MOI_tensor = np.eye(3)
    MOI_tensor *= MOI_off_axis
    MOI_tensor[axis_to_idx[rotation_axis], axis_to_idx[rotation_axis]] = MOI_on_axis
    return MOI_tensor

def get_MOI_tensor_string(tensor):
    return f'<inertia ixx="{tensor[0,  0]: 0.5f}" ixy="{tensor[0, 1] : 0.5f}" ixz="{tensor[0, 2] : 0.5f}"' + \
    f'iyy="{tensor[1, 1] : 0.5f}" iyz="{tensor[1, 2] : 0.5f}" izz="{tensor[2, 2] : 0.5f}"/>'


get_MOI_tensor_string(calculate_MOI_tensor(1, 0.03, 0.100))
get_MOI_tensor_string(calculate_MOI_tensor(0, 0, 0))




'<inertia ixx=" 0.00000" ixy=" 0.00000" ixz=" 0.00000"iyy=" 0.00000" iyz=" 0.00000" izz=" 0.00000"/>'

In [34]:
meshcat.Delete()

visualizer = ModelVisualizer(meshcat=meshcat)
visualizer.AddModels(
        "./models/singleAxisCatBot.urdf"
)

visualizer.Run(loop_once=False)

meshcat.DeleteAddedControls()

Click 'Stop Running' or press Esc to quit
